In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import torch
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
from tqdm import tqdm
import torch.nn.functional as F


In [28]:
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Apple GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA GPU")
else:
    device = torch.device('cpu')

Apple GPU


In [29]:
def getData(path):
    train_file = np.load(path+"/train.npz")
    train_data = train_file['data']
    test_file = np.load(path+"/test_input.npz")
    test_data = test_file['data']
    print(f"Training Data's shape is {train_data.shape} and Test Data's is {test_data.shape}")
    return train_data, test_data
trainData, testData = getData("./data/")

Training Data's shape is (10000, 50, 110, 6) and Test Data's is (2100, 50, 50, 6)


In [30]:
class WindowedNormalizedDataset(Dataset):
    def __init__(self, data, scale=10.0):
        self.data = data
        self.scale = scale
        self.dt = 0.1
        self.interaction_radius = 15.0  # Meters for local interaction

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx].copy()
        presence = (scene[..., 0] != 0) | (scene[..., 1] != 0)

        origin = scene[0, 49].copy()
        tx, ty, _, _, theta, _ = origin

        cos_theta = np.cos(-theta)
        sin_theta = np.sin(-theta)

        # Allocate space for all features (0-13 = 14 features total)
        normalized_scene = np.zeros((50, 110, 14), dtype=np.float32)

        # --- Existing normalization (features 0-10) ---
        # Position normalization
        x = scene[..., 0] - tx
        y = scene[..., 1] - ty
        x_n = x * cos_theta - y * sin_theta
        y_n = x * sin_theta + y * cos_theta
        normalized_scene[..., 0] = x_n / self.scale
        normalized_scene[..., 1] = y_n / self.scale

        # Velocity normalization
        vx = scene[..., 2]
        vy = scene[..., 3]
        vx_n = vx * cos_theta - vy * sin_theta
        vy_n = vx * sin_theta + vy * cos_theta
        normalized_scene[..., 2] = vx_n / self.scale
        normalized_scene[..., 3] = vy_n / self.scale

        # Heading normalization
        heading = scene[..., 4]
        normalized_heading = heading - theta
        normalized_heading = (normalized_heading + np.pi) % (2 * np.pi) - np.pi
        normalized_scene[..., 4] = normalized_heading

        # Agent type and presence
        normalized_scene[..., 5] = scene[..., 5]
        normalized_scene[..., 6] = presence.astype(np.float32)

        # Speed
        speed = np.sqrt(vx ** 2 + vy ** 2)
        normalized_scene[..., 7] = speed / self.scale

        # Distance to ego
        ego_pos = scene[0, :, :2]
        dist_to_ego = np.linalg.norm(scene[..., :2] - ego_pos[None, :, :], axis=-1)
        normalized_scene[..., 8] = dist_to_ego / self.scale

        # Acceleration
        accel = np.zeros_like(speed)
        accel[:, 1:] = (speed[:, 1:] - speed[:, :-1]) / self.dt
        accel[:, 0] = accel[:, 1]
        normalized_scene[..., 9] = accel / (self.scale / self.dt)

        # TTC
        rel_speed = np.sqrt((vx - vx[0:1])**2 + (vy - vy[0:1])**2)
        ttc = dist_to_ego / (rel_speed + 1e-5)
        ttc = np.clip(ttc, 0, 10)
        normalized_scene[..., 10] = ttc / 10.0

        # === NEW FEATURE 1: Dynamic Interaction Density (Optimized) ===
        positions = scene[..., :2]
        interaction_density = np.zeros((50, 110), dtype=np.float32)
        
        # Vectorized implementation
        for t in range(110):
            # Compute pairwise distances at timestep t
            dist_matrix = np.linalg.norm(positions[:, t, None] - positions[:, t], axis=-1)
            # Count neighbors within radius (excluding self)
            neighbor_counts = np.sum((dist_matrix < self.interaction_radius) & (dist_matrix > 0), axis=-1)
            interaction_density[:, t] = neighbor_counts
            
        normalized_scene[..., 11] = interaction_density / 20.0  # Normalize

        # === NEW FEATURE 2: Future Motion Context ===
        # Vector from current position to last observed position
        future_context = np.zeros((50, 110, 2), dtype=np.float32)
        
        for agent in range(50):
            valid_timesteps = np.where(presence[agent, :50])[0]
            if len(valid_timesteps) > 0:
                last_idx = valid_timesteps[-1]
                goal_vector = positions[agent, last_idx] - positions[agent]
                # Rotate to ego frame
                goal_x = goal_vector[..., 0] * cos_theta - goal_vector[..., 1] * sin_theta
                goal_y = goal_vector[..., 0] * sin_theta + goal_vector[..., 1] * cos_theta
                future_context[agent, :, 0] = goal_x / self.scale
                future_context[agent, :, 1] = goal_y / self.scale
                
        normalized_scene[..., 12] = future_context[..., 0]  # Relative goal X
        normalized_scene[..., 13] = future_context[..., 1]  # Relative goal Y

        # --- Masking ---
        missing_mask = np.expand_dims(~presence, -1)
        normalized_scene = np.where(missing_mask, 0, normalized_scene)

        # Inputs: first 50 timesteps
        X = normalized_scene[:, :50, :]  # Shape: (50 agents, 50 timesteps, 14 features)

        # Target: ego future positions and presence
        ego_future = normalized_scene[0, 50:]
        Y = np.zeros((60, 3), dtype=np.float32)
        Y[:, :2] = ego_future[:, :2]
        Y[:, 2] = ego_future[:, 6]  # presence

        return (
            torch.tensor(X, dtype=torch.float32),
            torch.tensor(Y, dtype=torch.float32),
            torch.tensor(origin, dtype=torch.float32)
        )

In [31]:
class WindowedNormalizedTestDataset(Dataset):
    def __init__(self, data, scale=10.0):
        self.data = data
        self.scale = scale
        self.dt = 0.1
        self.interaction_radius = 15.0  # Meters for local interaction

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx].copy()
        presence = (scene[..., 0] != 0) | (scene[..., 1] != 0)

        origin = scene[0, 49].copy()
        tx, ty, _, _, theta, _ = origin

        cos_theta = np.cos(-theta)
        sin_theta = np.sin(-theta)

        # Allocate space for all features (0-13 = 14 features total)
        normalized_scene = np.zeros((50, 50, 14), dtype=np.float32)

        # --- Existing normalization (features 0-10) ---
        # Position normalization
        x = scene[..., 0] - tx
        y = scene[..., 1] - ty
        x_n = x * cos_theta - y * sin_theta
        y_n = x * sin_theta + y * cos_theta
        normalized_scene[..., 0] = x_n / self.scale
        normalized_scene[..., 1] = y_n / self.scale

        # Velocity normalization
        vx = scene[..., 2]
        vy = scene[..., 3]
        vx_n = vx * cos_theta - vy * sin_theta
        vy_n = vx * sin_theta + vy * cos_theta
        normalized_scene[..., 2] = vx_n / self.scale
        normalized_scene[..., 3] = vy_n / self.scale

        # Heading normalization
        heading = scene[..., 4]
        normalized_heading = heading - theta
        normalized_heading = (normalized_heading + np.pi) % (2 * np.pi) - np.pi
        normalized_scene[..., 4] = normalized_heading

        # Agent type and presence
        normalized_scene[..., 5] = scene[..., 5]
        normalized_scene[..., 6] = presence.astype(np.float32)

        # Speed
        speed = np.sqrt(vx ** 2 + vy ** 2)
        normalized_scene[..., 7] = speed / self.scale

        # Distance to ego
        ego_pos = scene[0, :, :2]
        dist_to_ego = np.linalg.norm(scene[..., :2] - ego_pos[None, :, :], axis=-1)
        normalized_scene[..., 8] = dist_to_ego / self.scale

        # Acceleration
        accel = np.zeros_like(speed)
        accel[:, 1:] = (speed[:, 1:] - speed[:, :-1]) / self.dt
        accel[:, 0] = accel[:, 1]
        normalized_scene[..., 9] = accel / (self.scale / self.dt)

        # TTC
        rel_speed = np.sqrt((vx - vx[0:1])**2 + (vy - vy[0:1])**2)
        ttc = dist_to_ego / (rel_speed + 1e-5)
        ttc = np.clip(ttc, 0, 10)
        normalized_scene[..., 10] = ttc / 10.0

        # === NEW FEATURE 1: Dynamic Interaction Density (Optimized) ===
        positions = scene[..., :2]
        interaction_density = np.zeros((50, 50), dtype=np.float32)
        
        # Vectorized implementation
        for t in range(50):
            # Compute pairwise distances at timestep t
            dist_matrix = np.linalg.norm(positions[:, t, None] - positions[:, t], axis=-1)
            # Count neighbors within radius (excluding self)
            neighbor_counts = np.sum((dist_matrix < self.interaction_radius) & (dist_matrix > 0), axis=-1)
            interaction_density[:, t] = neighbor_counts
            
        normalized_scene[..., 11] = interaction_density / 20.0  # Normalize

        # === NEW FEATURE 2: Future Motion Context ===
        # Vector from current position to last observed position
        future_context = np.zeros((50, 50, 2), dtype=np.float32)
        
        for agent in range(50):
            valid_timesteps = np.where(presence[agent, :50])[0]
            if len(valid_timesteps) > 0:
                last_idx = valid_timesteps[-1]
                goal_vector = positions[agent, last_idx] - positions[agent]
                # Rotate to ego frame
                goal_x = goal_vector[..., 0] * cos_theta - goal_vector[..., 1] * sin_theta
                goal_y = goal_vector[..., 0] * sin_theta + goal_vector[..., 1] * cos_theta
                future_context[agent, :, 0] = goal_x / self.scale
                future_context[agent, :, 1] = goal_y / self.scale
                
        normalized_scene[..., 12] = future_context[..., 0]  # Relative goal X
        normalized_scene[..., 13] = future_context[..., 1]  # Relative goal Y

        # --- Masking ---
        missing_mask = np.expand_dims(~presence, -1)
        normalized_scene = np.where(missing_mask, 0, normalized_scene)

        # Inputs: first 50 timesteps
        X = normalized_scene[:, :50, :]  # Shape: (50 agents, 50 timesteps, 14 features)

        # Target: ego future positions and presence
        ego_future = normalized_scene[0, 50:]
        Y = np.zeros((60, 3), dtype=np.float32)
        Y[:, :2] = ego_future[:, :2]
        Y[:, 2] = ego_future[:, 6]  # presence

        return (
            torch.tensor(X, dtype=torch.float32),
            torch.tensor(Y, dtype=torch.float32),
            torch.tensor(origin, dtype=torch.float32)
        )

In [32]:
def denormalize_ego_batch(predicted, origin, scale=10.0):
    """
    Convert batch of normalized (and scaled) ego predictions back to global coordinates.

    predicted: (B, ..., 2) tensor of normalized [x, y] positions
    origin: (B, 6) tensor of ego's reference state at t=49
    Returns:
        (B, ..., 2) tensor of global [x, y] positions
    """
    tx = origin[:, 0]  # (B,)
    ty = origin[:, 1]  # (B,)
    theta = origin[:, 4]  # (B,)

    cos_theta = torch.cos(theta)
    sin_theta = torch.sin(theta)

    # Expand for broadcasting
    while len(cos_theta.shape) < len(predicted.shape) - 1:
        cos_theta = cos_theta.unsqueeze(1)
        sin_theta = sin_theta.unsqueeze(1)
        tx = tx.unsqueeze(1)
        ty = ty.unsqueeze(1)

    # Unscale before denormalizing
    x = predicted[..., 0] * scale
    y = predicted[..., 1] * scale

    # Rotate
    x_rot = x * cos_theta - y * sin_theta
    y_rot = x * sin_theta + y * cos_theta

    # Translate
    x_global = x_rot + tx
    y_global = y_rot + ty

    return torch.stack([x_global, y_global], dim=-1)


In [33]:
trainData[1, 0, 49, :], trainData[1, 0, 50, :]

(array([ 3.16906469e+03,  1.68248551e+03,  5.46145515e+00, -5.85380650e+00,
        -8.22467566e-01,  0.00000000e+00]),
 array([ 3.16959927e+03,  1.68191109e+03,  5.35655550e+00, -5.75120145e+00,
        -8.22600550e-01,  0.00000000e+00]))

In [34]:
data = WindowedNormalizedDataset(trainData)
X, Y, origin = data.__getitem__(1)
X[0, 49, :], Y[0, :], origin.shape

(tensor([ 0.0000,  0.0000,  0.8006,  0.0019,  0.0000,  0.0000,  1.0000,  0.8006,
          0.0000, -0.0057,  0.0000,  0.1000,  0.0000,  0.0000]),
 tensor([7.8468e-02, 9.1270e-05, 1.0000e+00]),
 torch.Size([6]))

In [35]:
# x, y = denormalize_ego(Y[0, :2], origin)
# x, y

In [36]:
import torch
import torch.nn as nn

class TrajectoryTransformer(nn.Module):
    def __init__(self, input_dim=700, model_dim=256, num_heads=8, num_layers=6, dropout=0.1, pred_len=60, num_agents=50):
        super().__init__()
        self.model_dim = model_dim
        self.pred_len = pred_len
        self.num_agents = num_agents
        
        # Process each agent's full trajectory (50*7 = 350) into a single token
        self.trajectory_encoder = nn.Sequential(
            nn.Linear(input_dim, model_dim),
            nn.LayerNorm(model_dim),
            nn.ReLU(),
            nn.Linear(model_dim, model_dim),
            nn.LayerNorm(model_dim),
            nn.ReLU(),
            nn.Linear(model_dim, model_dim),
            nn.LayerNorm(model_dim),
            nn.ReLU()
        )
        
        # 2-layer transformer encoder to process agent tokens
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=model_dim, 
                nhead=num_heads, 
                dropout=dropout, 
                batch_first=True
            ),
            num_layers=num_layers
        )
        
        self.output_fcpre = nn.Linear(model_dim, model_dim)  # 60*2 = 120
        self.output_fc = nn.Linear(model_dim, pred_len * 2)  # 60*2 = 120
    
    def forward(self, x):
        B, N, T, Ft = x.shape
        x = x.view(B, N, T * Ft)  # (B, 50, 350)
        agent_tokens = self.trajectory_encoder(x)  # (B, 50, model_dim)
        encoded_tokens = self.transformer_encoder(agent_tokens)  # (B, 50, model_dim)
        ego_token = encoded_tokens[:, 0, :]  # (B, model_dim)
        output = F.relu(self.output_fcpre(ego_token))  # (B, pred_len*2)
        output = self.output_fc(output)  # (B, pred_len*2)
        output = output.view(B, self.pred_len, 2)  # (B, 60, 2)
        
        return output

# Test run
model = TrajectoryTransformer()
x = torch.randn(1, 50, 50, 14)  
out = model(x)
print(f"Input shape: {x.shape}")
print(f"Output shape: {out.shape}")  # Expected: (1, 60, 2)

# Print model summary
print(f"\nModel parameters: {sum(p.numel() for p in model.parameters()):,}")

Input shape: torch.Size([1, 50, 50, 14])
Output shape: torch.Size([1, 60, 2])

Model parameters: 8,299,640


In [37]:
model = TrajectoryTransformer().to(device=device)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 8299640


In [38]:
np.random.seed(42)
num_samples = trainData.shape[0]
indices = np.random.permutation(num_samples)
split_index = int(0.9 * num_samples)
train_idx, val_idx = indices[:split_index], indices[split_index:]

# Split the data
train_data = trainData[train_idx]
val_data = trainData[val_idx]

print("Train shape:", train_data.shape)
print("Validation shape:", val_data.shape)

Train shape: (9000, 50, 110, 6)
Validation shape: (1000, 50, 110, 6)


In [39]:
trainTensor = WindowedNormalizedDataset(train_data)
testTensor = WindowedNormalizedDataset(val_data)
train_dataloader = DataLoader(trainTensor, batch_size=128, shuffle=True)
val_dataloader = DataLoader(testTensor, batch_size=128, shuffle=False)

In [40]:
#  train MSE 0.0148209710 | train val MSE 0.0955192002 | val MAE 4.9818070605 | val MSE 9.5519190058

torch.cuda.empty_cache()

best_model = torch.load("./models/modelI/best_model.pt")
model.load_state_dict(best_model)

epochs = 1000
lossFn = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.25)
best_val_loss = 0.0955192002 # float('inf')
best_train_loss = 0.0148209710 # float('inf')
position_scale = 1.0
velocity_scale = 1.0
all_losses = {
    'training_mse_loss':[],
    'validation_mse_loss':[],
    'true_mse':[],
    'true_mae':[]
}

for each_epoch in range(epochs):
    model.train()
    runningLoss = 0.0
    loop = tqdm(train_dataloader, desc=f"Epoch [{each_epoch+1}/{epochs}]")
    
    for batchX, batchY, origin in loop:
        batchX = batchX.to(device)
        batchY = batchY.to(device)
        origin = origin.to(device)

        
        pred = model(batchX)  # pred shape: (B, 60, 2)
        
        loss = lossFn(pred[..., :2], batchY[..., :2]).to(device)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        runningLoss += loss.item()        
    
    model.eval()
    val_loss = 0
    val_mae = 0
    val_mse = 0
    
    with torch.no_grad():
        for batchX, batchY, origin in loop:
            batchX = batchX.to(device)
            batchY = batchY.to(device)
            origin = origin.to(device)

            
            pred = model(batchX)  # pred shape: (B, 60, 2)
            
            loss = lossFn(pred[..., :2], batchY[..., :2]).to(device)
            unnorm_pred = denormalize_ego_batch(pred, origin)
            unnorm_true = denormalize_ego_batch(batchY, origin)

            # print(pred[..., :2].shape, batchY[..., :2].shape, origin.shape, unnorm_pred.shape)
            
            # break
            
            # unnorm_pred = denormalize_ego(pred[..., :2], origin)
            # unnorm_true = denormalize_ego(batchY, origin)

            
            val_loss += loss.item()
            val_mae += nn.L1Loss()(unnorm_pred[..., :2], unnorm_true[..., :2]).item()
            val_mse += nn.MSELoss()(unnorm_pred[..., :2], unnorm_true[..., :2]).item()
    # break
    train_loss = runningLoss/len(train_dataloader)
    val_loss /= len(val_dataloader)
    val_mae /= len(val_dataloader)
    val_mse /= len(val_dataloader)
    
    all_losses["training_mse_loss"].append(train_loss)
    all_losses["validation_mse_loss"].append(val_loss)
    all_losses["true_mse"].append(val_mse)
    all_losses["true_mae"].append(val_mae)
    
    loop.write(f" train MSE {train_loss:.10f} | train val MSE {val_loss:.10f} | val MAE {val_mae:.10f} | val MSE {val_mse:.10f}")
    scheduler.step()
    
    if train_loss < best_train_loss and val_loss < best_val_loss: # - 1e-3
        best_val_loss = val_loss
        best_train_loss = train_loss
        no_improvement = 0
        torch.save(model.state_dict(), "./models/modelI/best_model.pt")
        loop.write(f" model Saved")
    torch.cuda.empty_cache()


Epoch [1/1000]: 100%|██████████| 71/71 [01:03<00:00,  1.12it/s]


 train MSE 0.0492038001 | train val MSE 0.2322185440 | val MAE 8.0665035844 | val MSE 23.2218529880


Epoch [2/1000]: 100%|██████████| 71/71 [01:02<00:00,  1.14it/s]


 train MSE 0.0367420487 | train val MSE 0.2909632560 | val MAE 8.6918457448 | val MSE 29.0963252932


Epoch [3/1000]: 100%|██████████| 71/71 [01:01<00:00,  1.16it/s]


...

 train MSE 0.0291709699 | train val MSE 0.2392361572 | val MAE 7.7979869321 | val MSE 23.9236205518


Epoch [13/1000]: 100%|██████████| 71/71 [00:59<00:00,  1.19it/s]


 train MSE 0.0288806218 | train val MSE 0.2738061054 | val MAE 8.5432129279 | val MSE 27.3806151152


Epoch [14/1000]:  24%|██▍       | 17/71 [00:15<00:49,  1.09it/s]


KeyboardInterrupt: 

In [ ]:
test_dataset = WindowedNormalizedTestDataset(testData)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


best_model = torch.load("./models/modelI/best_model.pt")
model = model = TrajectoryTransformer().to(device=device)


model.load_state_dict(best_model)
model.eval()

pred_list = []
with torch.no_grad():
    for batchX, origin in test_loader:
        batchX = batchX.to(device)
        batchY = batchY.to(device)
        origin = origin.to(device)

        
        pred = model(batchX)  # pred shape: (B, 60, 2)
        
        unnorm_pred = denormalize_ego_batch(pred[..., :2], origin)
        # print(unnorm_pred.shape)
        pred_list.append(unnorm_pred.cpu().numpy())
        # print(len(pred))
        

pred_list = np.concatenate(pred_list, axis=0)  
pred_output = pred_list.reshape(-1, 2)  # (N*60, 2)
output_df = pd.DataFrame(pred_output, columns=['x', 'y'])
output_df.index.name = 'index'
output_df.to_csv('./models/final/testTransFormer.csv', index=True)